# OpenStreetMap Data Wrangling Project

In this notebook I summarize my steps and findings towards the Data Wrangling Project course, using OpenStreet Map as a Case Study.
This is in the Scope of the Udacity Nanodegree, P3: Wrangle OpenStreetMap Data

** Author **: [Marcelo Tyszler](mailto:tyszler.jobs@gmail.com)

** Start Date **: 04/Mar/2017

** Today's Date **: 04/Mar/2017

## Choice of tools

For this project I opted for:

* Keep basic documentation and notes in the Jupyter Notebook
* Save files in Github
* Use PyCharm as Python environment
* Use DB Browser for SQLite as SQL environment (in combination with command line)

